# COVID-19 data analysis

<img src=coronavirus-DNA-720.png  width="1200" >

In [35]:
#keep repository update
!git pull https://github.com/pcm-dpc/COVID-19.git master --no-edit

From https://github.com/pcm-dpc/COVID-19
 * branch            master     -> FETCH_HEAD
Already up-to-date.


In [36]:
#libraries
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import os
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
plt.style.use('ggplot')

In [37]:
#remove annoying worning about pandas assignment
pd.options.mode.chained_assignment = None

In [38]:
from sklearn.linear_model import LinearRegression,HuberRegressor,Ridge
lr = LinearRegression()

# Load national dataset

In [39]:
df = pd.read_csv(os.path.join('dati-andamento-nazionale',
                               'dpc-covid19-ita-andamento-nazionale.csv'))
df.data = pd.to_datetime(df.data)
#assuming terapia_intensiva (intensive care) and deceduti (deaths)
#are cumulative. It is not specified in the dataset.
#Computing Daily new cases
df['terapia_intensiva_change']=df['terapia_intensiva']-df['terapia_intensiva'].shift(1)
df['deceduti_daily']=df['deceduti']-df['deceduti'].shift(1)


#The number of deaths over the number 
#of new cases admitted in intensive care (daily)
df['not_admitted_ratio'] = df['deceduti_daily']/(df['terapia_intensiva_change'])
#The number of deaths minus the number 
#of new cases admitted in intensive care (daily)
df['not_admitted'] = df['deceduti_daily']-(df['terapia_intensiva_change'])

#remove unused
df.drop(['stato'],axis=1,inplace=True)
df.drop(['data'],axis=1).tail()

#compute log of selected columns
for c in ['nuovi_attualmente_positivi',#new positive case now
          'deceduti_daily',#today deaths
          'terapia_intensiva',#Intensive care
          'deceduti'#deaths
         ]:
    
    df[c+'_log']=np.log10(df[c])

## Fig1

In [40]:
yR='deceduti_daily'
df=df[~df[yR+'_log'].isna()]
#select before and after 
#the introduction of social distancing mesures
before = df[df['data']<'2020-03-9']
after = df[df['data']>'2020-03-9']

plt.style.use('ggplot')
fig,ax=plt.subplots(figsize=(8,8))

before.plot(x='data',
         y=yR+'_log',
         kind='scatter',ax=ax,c='b')

after.plot(x='data',
         y=yR+'_log',
         kind='scatter',ax=ax,c='r')

x=before['data'].values.reshape(-1, 1)
y=before[yR+'_log'].values
lr.fit(x,y)
#plt.plot(x, lr.predict(x), 'b-')
y_pred = lr.predict(before.data.values.astype(float).reshape(-1, 1))
before.loc[:,'y_pred']=y_pred

before.plot(x='data',
         y='y_pred',
         ax=ax,c='b',label='before')

x=after['data'].values.reshape(-1, 1)
y=after[yR+'_log'].values
lr.fit(x,y )
#plt.plot(x, lr.predict(x), 'b-')
y_pred = lr.predict(after.data.values.astype(float).reshape(-1, 1))
after.loc[:,'y_pred']=y_pred

after.plot(x='data',
         y='y_pred',
         ax=ax,c='r',label='after')
plt.legend(title='Social Distancing',fontsize=12)

#set ticks every week
ax.xaxis.set_major_locator(mdates.WeekdayLocator())
#set major ticks format
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))



#plt.xticks(rotation=70,ha='right')
plt.xlabel('Date', fontsize='16')
plt.ylabel('Today Minus Yesterday Deaths (log10)',fontsize='16')
plt.title('Death Trend',fontsize='18')
plt.savefig('Fig1.png')

## Fig2

In [31]:

plt.style.use('default')


fig,ax=plt.subplots(figsize=(8,4))
ax.plot(df['data'].dt.date, df['not_admitted_ratio'], "o",
        markersize=5, color='#007acc', alpha=0.9,label='Ratio')
ax.vlines(x=df['data'].dt.date, ymin=0, 
          ymax=df['not_admitted_ratio'], color='#007acc', alpha=0.2, linewidth=5)

#set ticks every week
ax.xaxis.set_major_locator(mdates.WeekdayLocator())
#set major ticks format
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
#designdate='March 08 2020 13:00'
#ax.axvline(pd.to_datetime(designdate), ymin=0.1,ymax=0.9,linewidth=2)

xtickslocs = ax.get_xticks()
xmin, xmax = ax.get_xlim()

ax.axhline(1, xmin=0.01,xmax=0.99,
           linewidth=2,color='r',alpha=0.6,label='Doom Line')

ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')

ymin, ymax = ax.get_ylim()
ax.spines['left'].set_bounds(0, ymax)


xmin, xmax = ax.get_xlim()
ax.spines['bottom'].set_bounds(xmin+1, xmax-1)

plt.xlabel('Date', fontsize='12')
plt.ylabel(' Deaths / Intensive Care Admissions', fontsize='12')
plt.title('Excluded From Intensive Care', fontsize='14')

plt.legend(loc='upper center', bbox_to_anchor=(1.1, 0.9), title='Legend')
plt.tight_layout()
plt.savefig('Fig2.png')
plt.show()

## Fig3

In [42]:
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
plt.style.use('default')
#df['data']=df['data'].dt.to_period('D')

fig,ax=plt.subplots(figsize=(8,4))
ax.plot(df['data'].dt.date, df['not_admitted'].clip(lower=0),
        "o", markersize=5, color='#007acc', alpha=0.9,label='Ratio')
ax.vlines(x=df['data'].dt.date, ymin=0, ymax=df['not_admitted'].clip(lower=0), 
          color='#007acc', alpha=0.2, linewidth=5)

#set ticks every week
ax.xaxis.set_major_locator(mdates.WeekdayLocator())
#set major ticks format
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
designdate='March 08 2020 13:00'
#ax.axvline(pd.to_datetime(designdate), ymin=0.1,ymax=0.9,linewidth=2)

xtickslocs = ax.get_xticks()
xmin, xmax = ax.get_xlim()


ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
#ax.spines['left'].set_smart_bounds(True)
#ax.spines['bottom'].set_smart_bounds(True)
ymin, ymax = ax.get_ylim()
ax.spines['left'].set_bounds(0, ymax)


xmin, xmax = ax.get_xlim()
ax.spines['bottom'].set_bounds(xmin+1, xmax-1)

plt.xlabel('Date', fontsize='12')
plt.ylabel('Deaths', fontsize='12')
plt.title('Excluded From Intensive Care', fontsize='14')

#plt.legend(loc='upper center', bbox_to_anchor=(1.1, 0.9), title='Legend')
plt.savefig('Fig3.png')
plt.show()

In [ ]:
!jupyter nbconvert --to html_toc DataVis.ipynb

[NbConvertApp] Converting notebook DataVis.ipynb to html_toc


In [34]:
!git add DataVis*
!git add -u
!git commit -m 'add comments to dataset'
!git push -u origin master

[master 9fd6195] add comments to dataset
 Committer: Michele Tinti <mtinti@ls29490.local>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly. Run the
following command and follow the instructions in your editor to edit
your configuration file:

    git config --global --edit

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 2 files changed, 78 insertions(+), 52 deletions(-)
Counting objects: 4, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.82 KiB | 0 bytes/s, done.
Total 4 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/mtinti/COVID-19.git
   dfa95e8..9fd6195  master -> master
Branch master set up to track remote branch master from origin.
